In [1]:
import matplotlib.pyplot as plt
import astropy
import os
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib as mpl
import pyarrow.parquet as pq
import matplotlib.image as mpimg
import webbrowser  
import time

from astropy import io
from astropy.io import fits
from astropy import table
from astropy import wcs

## Catalogue File

In [2]:
catalogue_file = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Catalogue/tractor-0260m062.fits'

In [3]:
catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))

_______

## Images

In [4]:
image_file_g = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/g-legacysurvey-0260m062-image.fits.fz'
image_file_r = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/r-legacysurvey-0260m062-image.fits.fz'
image_file_z = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/z-legacysurvey-0260m062-image.fits.fz'

________

## Create the new catalogue without point sources

In [5]:
def convert_tractor_catalogue(catalogue_file, image_file, image_name =''):
    """
    Converts a tractor fits file to a pandas dataframe to be given
    directly to an ImageDataset object.

    Parameters
    ----------
    catalogue_files : string
        tractor catalogue in fits table format 
    image_file:
        The image corresponding to this catalogue (to extract pixel information
        and naming information)
    """

    catalogue = astropy.table.Table(astropy.io.fits.getdata(catalogue_file,ignore_missing_end=True))

    dataframe = {}
    for name in catalogue.colnames:
        data = catalogue[name].tolist()
        dataframe[name] = data
    
    old_catalogue = pd.DataFrame(dataframe)
    hdul = astropy.io.fits.open(image_file)

    if len(image_name) == 0:
        original_image_ = image_file.split(os.path.sep)[-1]
        original_image = original_image_.replace(original_image_[:2],'')
    else:
        original_image = image_name
    
    #w = astropy.wcs.WCS(hdul[0].header, naxis=2)
    #x, y = w.wcs_world2pix(old_catalogue['ra'], old_catalogue['dec'], 1)
    
    new_catalogue = pd.DataFrame()
    new_catalogue['objid'] = old_catalogue['objid']
    new_catalogue['original_image'] = [original_image] * len(new_catalogue)
    new_catalogue['flux_g'] = old_catalogue['flux_g']
    new_catalogue['flux_r'] = old_catalogue['flux_r']
    new_catalogue['flux_z'] = old_catalogue['flux_z']
    new_catalogue['x'] = old_catalogue['bx'].astype('int')
    new_catalogue['y'] = old_catalogue['by'].astype('int')
    new_catalogue['ra'] = old_catalogue['ra']
    new_catalogue['dec'] = old_catalogue['dec']
    
    new_catalogue['type'] = old_catalogue['type']
    
    
    ## Removes PSFs
    catalogue_no_PSFs = new_catalogue[new_catalogue.type.astype("S") != b'PSF ']
    
    return catalogue_no_PSFs

__________

## Produce the catalogue

In [6]:
catalogue = convert_tractor_catalogue(catalogue_file, image_file_g)

________

## Sort the function according to flux

In [7]:
catalogue_sorted = catalogue.sort_values('flux_g', ascending=False)

_____

## Collect the cutouts from the website

#### WARNING: Can take a long time!

In [8]:
# Written like this to avoid accidentally running all the values 
#x = range(len(catalogue_sorted))

In [ ]:
for i in x:
    ra = catalogue_sorted['ra'].reset_index()
    dec = catalogue_sorted['dec'].reset_index()
    
    ra_value = ra.loc[i].ra
    dec_value = dec.loc[i].dec
    
    webbrowser.open(
        'http://legacysurvey.org/viewer/fits-cutout/?ra=' +str(ra_value) + 
        '&dec=' +str(dec_value) + '&layer=dr8-south&pixscale=0.27&bands=grz',
        new=0,
        autoraise=True
    )
    
    # Sleep function added to avoid overburderning browser
    time.sleep(0.5)